<a href="https://colab.research.google.com/github/mathemakitten/fastai-colab/blob/master/lesson1_homework_cats_dogs_redux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install kaggle CLI onto the server instance

2. Use the kaggle CLI to download the current data for the Dogs vs. Cats Redux competition
https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition 

3. Configure the new data to the file structure in the same way that was used in the sample lesson 1 notebook

4. Make a copy of the lesson 1 notebook and use the new copy to draw in the new Dogs Vs. Cats data (if you copy the notebook outside of the course folder, don't forget the utils.py, vgg26.py files etc)

5. Run the relevant code cells on the sample set of new Dogs v. Cats data to make a prediction on the new image data set.

6. Once the sample set works, modify the jupyter notebook to use the new test data images

7. Write a script that takes the predict() data of the new Dogs vs. Cats data and writes the data to a new csv file in the format of the sample_submission.csv file that was downloaded with the Dogs vs. Cats

8. Submit that new submission.csv file to the kaggle via the CLI tool

In [3]:
!rm utils.py*
!rm vgg16.py*
!rm vgg16bn.py*

!wget https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/utils.py
!wget https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/vgg16.py
!wget https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/vgg16bn.py
  
! pip install bcolz

from utils import *
from vgg16 import Vgg16

from shutil import copyfile
import zipfile
import os

%matplotlib inline

--2019-01-08 04:02:52--  https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7953 (7.8K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   7.77K  --.-KB/s    in 0s      

2019-01-08 04:02:52 (86.2 MB/s) - ‘utils.py’ saved [7953/7953]

--2019-01-08 04:02:53--  https://raw.githubusercontent.com/mathemakitten/fastai-colab/master/vgg16.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8755 (8.5K) [text/plain]
Saving to: ‘vgg16.py’

vgg16

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
! cp /content/drive/'My Drive'/'Colab Notebooks'/fastai/kaggle.json /root/.kaggle/
! kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

test.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
! ls 
! mkdir data

data   __pycache__  sample_submission.csv  train.zip  vgg16bn.py
drive  sample_data  test.zip		   utils.py   vgg16.py
mkdir: cannot create directory ‘data’: File exists


In [7]:
MAIN_DIR = os.getcwd()
DATA_DIR = MAIN_DIR + '/data'

% cd $DATA_DIR

/content/data


In [8]:
if (os.path.exists('test') is False) or (os.path.exists('train') is False):
  for file in ("../test.zip", "../train.zip"):
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall('')
    zip_ref.close()
  print("Files unzipped")
else: 
  print("Files already unzipped")

Files already unzipped


In [9]:
# Check the files in the directories unzipped properly
!ls train | head -5

cats
dogs


In [10]:
# Determine how many training samples we have - should be 25,000
# A 20% validation set of 25,000 would be 5,000
!ls train | wc -l

2


In [14]:
# Setup directory structure for data

!mkdir valid
!mkdir results
!mkdir -p sample/train
!mkdir -p sample/valid
!mkdir -p sample/test

mkdir: cannot create directory ‘valid’: File exists
mkdir: cannot create directory ‘results’: File exists


In [23]:
# Create a 20% validation set - 5000 images

# Get all the training images
% cd $DATA_DIR
% cd train

imgs = glob('*.jpg')
shuffled = np.random.permutation(imgs)

for i in range(5000): 
  os.rename(shuffled[i], DATA_DIR + '/valid/' + shuffled[i]) # recall that DATA_DIR = /content/data

/content/data
/content/data/train
cats/  dogs/


IndexError: ignored

In [0]:
# Create a sample training dataset by copying over a small 1% sample (200 images)

sample_train_imgs = glob('*.jpg')
shuffled_train_sample = np.random.permutation(sample_train_imgs)

for i in range(200): 
  copyfile(shuffled_train_sample[i], DATA_DIR + '/sample/train/' + shuffled_train_sample[i])

In [0]:
# As above, create a sample validation dataset for 50 validation images 

% cd $DATA_DIR
% cd valid

sample_valid_imgs = glob('*.jpg')
shuffled_valid_sample = np.random.permutation(sample_valid_imgs)

for i in range(50): 
  copyfile(shuffled_valid_sample[i], DATA_DIR + '/sample/valid/' + shuffled_valid_sample[i])

In [0]:
# Create folders for cats vs. dogs in both the full & sample datasets

% cd $DATA_DIR/sample/train
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

% cd $DATA_DIR/sample/valid
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

% cd $DATA_DIR/valid
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

% cd $DATA_DIR/train
% mkdir cats
% mkdir dogs
% mv cat.*.jpg cats/
% mv dog.*.jpg dogs/

In [0]:
# Test set: create an 'unknown' label for given test set because we're not sure if cat or dog

% cd $DATA_DIR/test
% mkdir unknown
% mv *.jpg unknown/

In [24]:
# Start model training
# Set up where to find the datasets

% cd $DATA_DIR

# Run pipeline on sample to ensure it's working
path = DATA_DIR + '/' # '/sample/'; otherwise just + '/' 

test_path = DATA_DIR + '/test/'
results_path = DATA_DIR + '/results/'

train_path = path + '/train/'
valid_path = path + '/valid/'

/content/data


In [25]:
# Instantiate vgg16 class
vgg = Vgg16()

/content/vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/content/vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/content/vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
/content/vgg16.py:100: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


In [0]:
# Set hyperparameters
batch_size=64
num_epochs=3
#vgg.model.optimizer.lr = 0.01 

In [27]:
# Run model finetuning to switch down to only 2 classes (cats & dogs)

batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size) # batch_size=batch_size*2
vgg.finetune(batches)

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [28]:
# Run model training to fit the model
# Training: 20000 observations / 64 batch size = 312 groups

for epoch in range(num_epochs):
  print("Running epoch: %d" % epoch)
  vgg.fit(batches, val_batches, nb_epoch=num_epochs)
  current_weights = 'ft%d.h5' % epoch # save weights for every pass in a .h5 file format
  vgg.model.save_weights(results_path + current_weights) # note: should switch this to persistent storage on Google Drive
print("Fit the model over " + str(num_epochs) + " epochs")

Running epoch: 0
Epoch 1/3
312/312 [==============================] - 248s 796ms/step - loss: 0.6814 - acc: 0.6275 - val_loss: 0.6166 - val_acc: 0.6701
Epoch 2/3
312/312 [==============================] - 245s 785ms/step - loss: 0.6233 - acc: 0.6736 - val_loss: 0.5511 - val_acc: 0.6971
Epoch 3/3
312/312 [==============================] - 241s 772ms/step - loss: 0.6056 - acc: 0.6901 - val_loss: 0.4856 - val_acc: 0.7743
Running epoch: 1
Epoch 1/3
312/312 [==============================] - 241s 773ms/step - loss: 0.5928 - acc: 0.6971 - val_loss: 0.4821 - val_acc: 0.7585
Epoch 2/3
312/312 [==============================] - 241s 771ms/step - loss: 0.6029 - acc: 0.6936 - val_loss: 0.5266 - val_acc: 0.7243
Epoch 3/3
312/312 [==============================] - 241s 771ms/step - loss: 0.5975 - acc: 0.7015 - val_loss: 0.5831 - val_acc: 0.6937
Running epoch: 2
Epoch 1/3
312/312 [==============================] - 241s 772ms/step - loss: 0.6006 - acc: 0.6951 - val_loss: 0.4804 - val_acc: 0.7607
Epoc

TypeError: ignored

In [0]:
# HN to be continued from here. 
# TODO: Assess if fit_generator is working as expected in Keras 2 
# Find out why performance is so low